<a href="https://colab.research.google.com/github/WilhenHS/Day-7-Portfolio-Individual-de-Recomendacao/blob/main/Day_7_Portf%C3%B3lio_Individual_Recomenda%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 7 - Portfólio Individual - Recomendação

## 1 - Importação das bibliotecas que serão utilizadas

In [1]:
#Importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

## 2 - Carregando e visualizando os dados

### 2.1 - Carregando os dados

In [2]:
# Carregamento dos arquivos adicionados manualmente ao colab
order_items = pd.read_csv('DNC_order_items_dataset.csv')
order_reviews = pd.read_csv('DNC_order_reviews_dataset.csv')
orders = pd.read_csv('DNC_orders_dataset.csv')
product = pd.read_csv('DNC_products_dataset.csv')

### 2.2 - Visualização dos dados

In [3]:
order_items.head()

,Unnamed: 0,order_id,order_item_id,product_id,price
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90
1,1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90
2,2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00
3,3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90


In [4]:
order_reviews.head()

,Unnamed: 0,review_id,order_id,review_score
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [5]:
orders.head()

,Unnamed: 0,order_id,customer_id,order_status
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [6]:
product.head()

,Unnamed: 0,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,225.0,16.0,10.0,14.0
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1000.0,30.0,18.0,20.0
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,154.0,18.0,9.0,15.0
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,371.0,26.0,4.0,26.0
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,625.0,20.0,17.0,13.0


## 3 - Explorando os dados

In [7]:
# Validando valores nulos
print(orders.isnull().sum(),'\n\n')
print(product.isnull().sum(),'\n\n')
print(order_reviews.isnull().sum(),'\n\n')
print(order_items.isnull().sum(),'\n\n')

Unnamed: 0      0
order_id        0
customer_id     0
order_status    0
dtype: int64 


Unnamed: 0                      0
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64 


Unnamed: 0      0
review_id       0
order_id        0
review_score    0
dtype: int64 


Unnamed: 0       0
order_id         0
order_item_id    0
product_id       0
price            0
dtype: int64 




Há produtos com valores importantes como nulos, como o tamanho do nome e o nome da categoria. Daremos uma olhada mais aproximada neles:

In [8]:
# Visualizando os valores nulos do conjunto de dados product
product[product['product_category_name'].isna()]

,Unnamed: 0,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
32515,32515,b0a0c5dd78e644373b199380612c350a,NaN,NaN,NaN,1800.0,30.0,20.0,70.0
32589,32589,10dbe0fbaa2c505123c17fdc34a63c56,NaN,NaN,NaN,800.0,30.0,10.0,23.0
32616,32616,bd2ada37b58ae94cc838b9c0569fecd8,NaN,NaN,NaN,200.0,21.0,8.0,16.0
32772,32772,fa51e914046aab32764c41356b9d4ea4,NaN,NaN,NaN,1300.0,45.0,16.0,45.0


Estes produtos não possuem informações relevantes para nosso estudo e tão pouco para um cliente realizar uma compra (Produto não se encontra em nenhuma categoria específica e nem mesmo contém nome), desta forma, iremos removê-los do conjunto de dados para não comprometer nossa análise.

Sugestão para a equipe de cadastro de produtos analisar se houve algum problema no cadastro destes produtos, e/ou para que a equipe que extraiu os dados, realize uma nova extração de dados se atentando para possíveis problemas durante a extração.

In [9]:
# Removendo os valores nulos do conjunto de dados product
product = product.dropna()

In [10]:
# Quantidade de produtos por categoria
product['product_category_name'].value_counts()

cama_mesa_banho                  3029
esporte_lazer                    2867
moveis_decoracao                 2657
beleza_saude                     2444
utilidades_domesticas            2335
                                 ... 
casa_conforto_2                     5
fashion_roupa_infanto_juvenil       5
pc_gamer                            3
seguros_e_servicos                  2
cds_dvds_musicais                   1
Name: product_category_name, Length: 73, dtype: int64

In [11]:
# Produtos mais comprados
order_items['product_id'].value_counts()

aca2eb7d00ea1a7b8ebd4e68314663af    527
99a4788cb24856965c36a24e339b6058    488
422879e10f46682990de24d770e7f83d    484
389d119b48cf3043d311335e499d9c6b    392
368c6c730842d78016ad823897a372db    388
                                   ... 
c74e157255c4861775b6885cc14a1ced      1
71ca28a845714b71b378d81e93f0b183      1
53fda2e4872a41ed984770d4c60a5211      1
dbda643f8da1758793ef0dab19c18688      1
6c0163ecd7df58f4240f9a83ca6079c7      1
Name: product_id, Length: 32951, dtype: int64

In [12]:
# Pedidos mais avaliados
order_reviews['order_id'].value_counts()

c88b1d1b157a9999ce368f218a407141    3
8e17072ec97ce29f0e1f111e598b0c85    3
df56136b8031ecd28e200bb18e6ddb2e    3
03c939fd7fd3b38f8485a0f95798f1f6    3
66f943fd426ac8716ef6b6b97ace8bd8    2
                                   ..
6fb8f79677df0623bc0a613ccea59105    1
d8a23906d95c53fd16569067cefb9103    1
0560690f57123e9331f5731479b2d505    1
b9fffd14dc2869d69f441ff3c8f34bab    1
e3071b7624445af6e4f3a1b23718667d    1
Name: order_id, Length: 98673, dtype: int64

In [13]:
# Status dos pedidos
orders['order_status'].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

## 4 - Preparando os dados

In [14]:
# Removendo a primeira coluna dos datasets por não conter dados relevantes.
orders = orders.drop(orders.columns[0], axis=1)
product = product.drop(product.columns[0], axis=1)
order_reviews = order_reviews.drop(order_reviews.columns[0], axis=1)
order_items = order_items.drop(order_items.columns[0], axis=1)

In [15]:
# Unificando os dataframes
df = pd.merge(orders, order_reviews, on="order_id")
df = pd.merge(df, order_items, on="order_id")
df = pd.merge(df, product, on="product_id")

In [16]:
# Considerando apenas pedidos já entregues, pois indicam o sucesso de todo o procedimento de compra.
df_delivered = df[df['order_status']=='delivered']
df_delivered.head()

,order_id,customer_id,order_status,review_id,review_score,order_item_id,product_id,price,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,a54f0611adc9ed256b57ede6b6eb5114,4,1,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,40.0,268.0,500.0,19.0,8.0,13.0
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,b46f1e34512b0f4c74a72398b03ca788,4,1,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,40.0,268.0,500.0,19.0,8.0,13.0
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,dc90f19c2806f1abba9e72ad3c350073,5,1,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,40.0,268.0,500.0,19.0,8.0,13.0
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,delivered,1bafb430e498b939f258b9c9dbdff9b1,3,1,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,40.0,268.0,500.0,19.0,8.0,13.0
4,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,8d5266042046a06655c8db133d120ba5,4,1,595fac2a385ac33a80bd5114aec74eb8,118.70,perfumaria,29.0,178.0,400.0,19.0,13.0,19.0


In [17]:
# Validando se há algum valor nulo no conjunto de dados
df_delivered.isnull().sum()

order_id                      0
customer_id                   0
order_status                  0
review_id                     0
review_score                  0
order_item_id                 0
product_id                    0
price                         0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64

## 5 - Filtros Colaborativos

In [18]:
# Validando a média de reviews dos produtos
media_avaliacoes = pd.DataFrame(df_delivered.groupby('product_id')['review_score'].mean()).reset_index()
media_avaliacoes = media_avaliacoes.rename(columns = {'review_score':'media_avaliacoes'}, inplace = False)

# Validando a quantidade de reviews dos produtos
quant_avaliacoes = pd.DataFrame(df_delivered.groupby('product_id')['review_score'].count()).reset_index()
quant_avaliacoes = quant_avaliacoes.rename(columns = {'review_score':'quant_avaliacoes'}, inplace = False)

# Agrupando os dados pelo product_id
popular_products = pd.merge(media_avaliacoes, quant_avaliacoes[['product_id','quant_avaliacoes']], on="product_id")
popular_products = pd.merge(popular_products, product[['product_id','product_category_name']], on="product_id")

In [19]:
# Visualizando os top 10 produtos com maior avaliação média
popular_products.sort_values(by='media_avaliacoes', ascending=False).head(10)

,product_id,media_avaliacoes,quant_avaliacoes,product_category_name
0,00066f42aeeb9f3007548bb9d3f33c38,5.0,1,perfumaria
16793,8852eb03b04ec3268a66e9b696e25f6f,5.0,1,papelaria
16786,883e57b1f0fab96202d9e39efbe3b671,5.0,2,automotivo
16784,8839acf227b922877d3c49fe5267af61,5.0,3,automotivo
16777,88159bac10e3d4f4b69b157406c7f70f,5.0,1,esporte_lazer
16771,881218c6167982e1dd94cb98a085cbb9,5.0,1,eletrodomesticos
16770,880f4cc8f0bd44fe5afa9fc978a216e1,5.0,1,cine_foto
16769,880e2c1d5c4b60504e6b0a0c8bf89a1f,5.0,3,alimentos_bebidas
16767,880a54b4d8c28e6f7f519ae8caaa09db,5.0,1,cama_mesa_banho
16766,8805dd9f1994bee68f2e1351c5e64f26,5.0,1,relogios_presentes


In [20]:
# Visualizando os top 10 produtos com maior quantidade de avaliações
popular_products.sort_values(by='quant_avaliacoes', ascending=False).head(10)

,product_id,media_avaliacoes,quant_avaliacoes,product_category_name
21146,aca2eb7d00ea1a7b8ebd4e68314663af,4.048356,517,moveis_decoracao
8232,422879e10f46682990de24d770e7f83d,3.946502,486,ferramentas_jardim
18864,99a4788cb24856965c36a24e339b6058,3.921610,472,cama_mesa_banho
7038,389d119b48cf3043d311335e499d9c6b,4.133676,389,ferramentas_jardim
6765,368c6c730842d78016ad823897a372db,3.922680,388,ferramentas_jardim
10343,53759a2ecddad2bb87a079a1f1519f73,3.868633,373,ferramentas_jardim
25852,d1c427060a0f73f6b889a5c7c61f2ac4,4.284848,330,informatica_acessorios
10369,53b36df67ebb7c41585e8d54d6772e08,4.200627,319,relogios_presentes
2675,154e7e31ebfa092203795c972e5804a6,4.373239,284,beleza_saude
7697,3dd2a17168ec895c781a9191c1e95ad7,4.221402,271,informatica_acessorios


Como podemos ver, há muitos produtos que possuem uma média de avaliações alta (5), entretanto não tiveram uma quantidade de avaliações altas, o que pode fazer com que o modelo recomende produtos que não possuem tão alta relevância.

Já para os produtos com maior quantidade de avaliações, olhando para os dados, conseguimos validar que estes possuem uma relevância maior, por terem sido mais comprados e avaliados.

In [21]:
# Criando um score simples de média x quantidade
popular_products['score_avaliacoes'] = popular_products['quant_avaliacoes'] * popular_products['media_avaliacoes']

In [22]:
# Visualizando os top 10 produtos com melhor score de avaliacoes
popular_products.sort_values(by='score_avaliacoes', ascending=False).head(10)

,product_id,media_avaliacoes,quant_avaliacoes,product_category_name,score_avaliacoes
21146,aca2eb7d00ea1a7b8ebd4e68314663af,4.048356,517,moveis_decoracao,2093.0
8232,422879e10f46682990de24d770e7f83d,3.946502,486,ferramentas_jardim,1918.0
18864,99a4788cb24856965c36a24e339b6058,3.921610,472,cama_mesa_banho,1851.0
7038,389d119b48cf3043d311335e499d9c6b,4.133676,389,ferramentas_jardim,1608.0
6765,368c6c730842d78016ad823897a372db,3.922680,388,ferramentas_jardim,1522.0
10343,53759a2ecddad2bb87a079a1f1519f73,3.868633,373,ferramentas_jardim,1443.0
25852,d1c427060a0f73f6b889a5c7c61f2ac4,4.284848,330,informatica_acessorios,1414.0
10369,53b36df67ebb7c41585e8d54d6772e08,4.200627,319,relogios_presentes,1340.0
2675,154e7e31ebfa092203795c972e5804a6,4.373239,284,beleza_saude,1242.0
7697,3dd2a17168ec895c781a9191c1e95ad7,4.221402,271,informatica_acessorios,1144.0


## 6 - Matriz de Recomendação

In [23]:
# Definindo os métodos

def start_pipeline(dataf):
    return dataf.copy()

def get_product_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'order_id': pd.Series.nunique}) \
    .rename(columns={'order_id': 'order_id_count'}) \
    .set_index('product_id') \
    .sort_values('order_id_count', ascending = False)

def get_avg_ratings(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'score_avaliacoes': np.mean}) \
    .rename(columns={'score_avaliacoes': 'score_avaliacoes_avg'}) \
    .set_index('product_id') \
    .sort_values('score_avaliacoes_avg', ascending = False)

def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()

top_product_ratings = popular_products \
  .pipe(start_pipeline) \
  .pipe(get_avg_ratings) \
  .pipe(get_n_top_values, n=1500)

top_products_sellers = df_delivered \
  .pipe(start_pipeline) \
  .pipe(get_product_counts) \
  .pipe(get_n_top_values, n=10)

In [24]:
top_product_ratings[:10]

['aca2eb7d00ea1a7b8ebd4e68314663af',
 '422879e10f46682990de24d770e7f83d',
 '99a4788cb24856965c36a24e339b6058',
 '389d119b48cf3043d311335e499d9c6b',
 '368c6c730842d78016ad823897a372db',
 '53759a2ecddad2bb87a079a1f1519f73',
 'd1c427060a0f73f6b889a5c7c61f2ac4',
 '53b36df67ebb7c41585e8d54d6772e08',
 '154e7e31ebfa092203795c972e5804a6',
 '3dd2a17168ec895c781a9191c1e95ad7']

In [25]:
# Listagem os produtos mais 
top_products_sellers

['99a4788cb24856965c36a24e339b6058',
 'aca2eb7d00ea1a7b8ebd4e68314663af',
 '422879e10f46682990de24d770e7f83d',
 'd1c427060a0f73f6b889a5c7c61f2ac4',
 '389d119b48cf3043d311335e499d9c6b',
 '53b36df67ebb7c41585e8d54d6772e08',
 '368c6c730842d78016ad823897a372db',
 '53759a2ecddad2bb87a079a1f1519f73',
 '154e7e31ebfa092203795c972e5804a6',
 '2b4609f8948be18874494203496bc318']

In [26]:
# Produtos com as melhores avaliações
popular_products \
  .pipe(start_pipeline) \
  .pipe(get_avg_ratings)

,score_avaliacoes_avg
product_id,
aca2eb7d00ea1a7b8ebd4e68314663af,2093.0
422879e10f46682990de24d770e7f83d,1918.0
99a4788cb24856965c36a24e339b6058,1851.0
389d119b48cf3043d311335e499d9c6b,1608.0
368c6c730842d78016ad823897a372db,1522.0
...,...
69ae578cb2e84390bb7c44216380ad5e,1.0
1ec5d3bb50a489c6ac8923101492c197,1.0
69bfbc73c85e0c111060d84260d5fd64,1.0


In [27]:
# Definindo a matriz de relacionamento e pivotando
matriz_de_relacionamento_top_product_ratings = popular_products[popular_products["product_id"].isin(top_product_ratings)] \
  .pivot_table(index="product_id", columns="product_category_name", values="score_avaliacoes").fillna(0)

In [28]:
# Visualizando a matriz de relacionamento
matriz_de_relacionamento_top_product_ratings

product_category_name,agro_industria_e_comercio,alimentos,alimentos_bebidas,artes,artigos_de_natal,audio,automotivo,bebes,bebidas,beleza_saude,brinquedos,cama_mesa_banho,casa_conforto,casa_conforto_2,casa_construcao,cds_dvds_musicais,cine_foto,climatizacao,consoles_games,construcao_ferramentas_construcao,construcao_ferramentas_ferramentas,construcao_ferramentas_iluminacao,construcao_ferramentas_jardim,construcao_ferramentas_seguranca,cool_stuff,eletrodomesticos,eletrodomesticos_2,eletronicos,eletroportateis,esporte_lazer,fashion_bolsas_e_acessorios,fashion_calcados,fashion_roupa_masculina,ferramentas_jardim,flores,industria_comercio_e_negocios,informatica_acessorios,instrumentos_musicais,livros_interesse_geral,livros_tecnicos,malas_acessorios,market_place,moveis_colchao_e_estofado,moveis_cozinha_area_de_servico_jantar_e_jardim,moveis_decoracao,moveis_escritorio,moveis_sala,papelaria,pcs,perfumaria,pet_shop,portateis_casa_forno_e_cafe,relogios_presentes,tablets_impressao_imagem,telefonia,telefonia_fixa,utilidades_domesticas
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001b72dfd63e9833e8c02742adf472e3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00250175f79f584c14ab5cecd80553cd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0
005030ef108f58b46b78116f754d8d38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
008cff0e5792219fae03e570f980b330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00ba6d766f0b1d7b78a5ce3e1e033263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff29d8cb1cd0cd5ea37b80dac9939e1c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ff5f8606556b625613f069a5a9a48966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffa7e0cbe11656d11a117b534bb1db27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 7 - Usando a Similaridade de Cossenos para recomendação

In [29]:
# Definindo as variáveis utilizando os dados já tratados
cos_products = cosine_similarity(matriz_de_relacionamento_top_product_ratings)

# Visualizando os dados
print(cos_products)
cos_products.shape

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]


(1500, 1500)

In [30]:
# Definindo o método de recomendação.

def recomendar(product_id):
  print(f"Produto de interesse: {product_id}")
  try:
    produto_idx = matriz_de_relacionamento_top_product_ratings.index.tolist().index(product_id)
    closest_10_products = np.argsort(-cos_products[produto_idx])[1:11]
    print(f"Retornando 10 produtos mais próximos de: {product_id}.")
    return matriz_de_relacionamento_top_product_ratings.index[closest_10_products].tolist()
  except ValueError as e:
    print(f"{product_id}, não está incluso na matriz de recomendação. Retornando os 10 mais avaliados")
    return top_products_sellers

In [31]:
# Rodando o método de recomendação (Com ID de produto localizado)
id_produto_interesse = "ff29d8cb1cd0cd5ea37b80dac9939e1c"
ids_10_mais_proximos = recomendar(id_produto_interesse)

ids_10_mais_proximos

Produto de interesse: ff29d8cb1cd0cd5ea37b80dac9939e1c
Retornando 10 produtos mais próximos de: ff29d8cb1cd0cd5ea37b80dac9939e1c.


['6ab6d9589ae19abcee458918d4779fae',
 '50fd2b788dc166edd20512370dac54df',
 '7ae8d9b8eca8a6daf066e53ddc7034d3',
 '2002fc3ec2d8a30a7588628fe7aff958',
 'f51dde119b9a90e3c7464f0cc45c0953',
 '474bb6b54fc608ca71059a6c4f7ecda3',
 'a5f6ba3fe91a2e73f10b612131cb2404',
 '6a8631b72a2f8729b91514db87e771c0',
 'f89d2b130b571d24532514dca6fee2a8',
 'f89cd865cac300a9bf1320dd8f0fa223']

In [32]:
# Visualizando os produtos que foram recomendados para o caso de um produto localizado na base de dados:
popular_products[popular_products['product_id'].isin(ids_10_mais_proximos)].sort_values(by='score_avaliacoes', ascending=False)

,product_id,media_avaliacoes,quant_avaliacoes,product_category_name,score_avaliacoes
13157,6a8631b72a2f8729b91514db87e771c0,4.709677,62,eletronicos,292.0
10035,50fd2b788dc166edd20512370dac54df,3.825000,40,eletronicos,153.0
8859,474bb6b54fc608ca71059a6c4f7ecda3,3.222222,36,eletronicos,116.0
20351,a5f6ba3fe91a2e73f10b612131cb2404,3.133333,30,eletronicos,94.0
4069,2002fc3ec2d8a30a7588628fe7aff958,3.764706,17,eletronicos,64.0
13178,6ab6d9589ae19abcee458918d4779fae,3.812500,16,eletronicos,61.0
15234,7ae8d9b8eca8a6daf066e53ddc7034d3,4.500000,12,eletronicos,54.0
30166,f51dde119b9a90e3c7464f0cc45c0953,4.500000,12,eletronicos,54.0
30581,f89d2b130b571d24532514dca6fee2a8,4.416667,12,eletronicos,53.0
30580,f89cd865cac300a9bf1320dd8f0fa223,3.916667,12,eletronicos,47.0


In [33]:
# Rodando o método de recomendação (Com ID de produto não localizado)
id_produto_novo = "id_novo"
ids_10_mais_vendidos = recomendar(id_produto_novo)

ids_10_mais_vendidos

Produto de interesse: id_novo
id_novo, não está incluso na matriz de recomendação. Retornando os 10 mais avaliados


['99a4788cb24856965c36a24e339b6058',
 'aca2eb7d00ea1a7b8ebd4e68314663af',
 '422879e10f46682990de24d770e7f83d',
 'd1c427060a0f73f6b889a5c7c61f2ac4',
 '389d119b48cf3043d311335e499d9c6b',
 '53b36df67ebb7c41585e8d54d6772e08',
 '368c6c730842d78016ad823897a372db',
 '53759a2ecddad2bb87a079a1f1519f73',
 '154e7e31ebfa092203795c972e5804a6',
 '2b4609f8948be18874494203496bc318']

---

In [34]:
# Visualizando os produtos que foram recomendados para o caso de um produto não localizado na base de dados:
popular_products[popular_products['product_id'].isin(ids_10_mais_vendidos)].sort_values(by='score_avaliacoes', ascending=False)

,product_id,media_avaliacoes,quant_avaliacoes,product_category_name,score_avaliacoes
21146,aca2eb7d00ea1a7b8ebd4e68314663af,4.048356,517,moveis_decoracao,2093.0
8232,422879e10f46682990de24d770e7f83d,3.946502,486,ferramentas_jardim,1918.0
18864,99a4788cb24856965c36a24e339b6058,3.921610,472,cama_mesa_banho,1851.0
7038,389d119b48cf3043d311335e499d9c6b,4.133676,389,ferramentas_jardim,1608.0
6765,368c6c730842d78016ad823897a372db,3.922680,388,ferramentas_jardim,1522.0
10343,53759a2ecddad2bb87a079a1f1519f73,3.868633,373,ferramentas_jardim,1443.0
25852,d1c427060a0f73f6b889a5c7c61f2ac4,4.284848,330,informatica_acessorios,1414.0
10369,53b36df67ebb7c41585e8d54d6772e08,4.200627,319,relogios_presentes,1340.0
2675,154e7e31ebfa092203795c972e5804a6,4.373239,284,beleza_saude,1242.0
5442,2b4609f8948be18874494203496bc318,4.117871,263,beleza_saude,1083.0
